In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from bs4 import BeautifulSoup
import pandas as pd
import requests
import spacy
import re

def webscrapping(url):
    autor = []
    respuesta = requests.get(url)
    soup = BeautifulSoup(respuesta.text, 'html.parser')
    titulos_h3 = soup.find_all('h3')
    for titulo in titulos_h3:
        if titulo.text[0].isdigit() :
            autor.append(titulo.text)
    return autor

In [89]:
jk_rowling = webscrapping('https://psicologiaymente.com/reflexiones/frases-jk-rowling')
allan_poe = webscrapping('https://psicologiaymente.com/reflexiones/frases-edgar-allan-poe')
sherlock_holmes = webscrapping('https://psicologiaymente.com/reflexiones/frases-sherlock-holmes')

# jk_rowling = jk_rowling[:50]
# allan_poe = allan_poe[:50]
# sherlock_holmes = sherlock_holmes[:50]

In [90]:
def remove_regex(texto):
    return re.sub('\d+\.\s', '', texto)

jk_rowling = list(map(remove_regex, jk_rowling))
allan_poe = list(map(remove_regex, allan_poe))
sherlock_holmes = list(map(remove_regex, sherlock_holmes))

In [91]:
df_jk_rowling = pd.DataFrame(jk_rowling, columns=['frases'])
df_jk_rowling['autor'] = 'jk_rowling'

df_allan_poe = pd.DataFrame(allan_poe, columns=['frases'])
df_allan_poe['autor'] = 'allan_poe'

df_sherlock_holmes = pd.DataFrame(sherlock_holmes, columns=['frases'])
df_sherlock_holmes['autor'] = 'sherlock_holmes'


In [92]:
df = pd.concat([df_jk_rowling, df_allan_poe, df_sherlock_holmes], ignore_index=True)

In [93]:
df

,frases,autor
0,El fracaso implicó deshacerse de todo lo innec...,jk_rowling
1,No todo consiste en firmar libros y fotos publ...,jk_rowling
2,Todavía no me he olvidado de lo que se siente ...,jk_rowling
3,Cualquier cosa es posible si tiene suficiente ...,jk_rowling
4,La esperanza es algo que se mantiene para siem...,jk_rowling
...,...,...
265,"Anderson, no hables en voz alta. Reduces el co...",sherlock_holmes
266,El sentimiento es un defecto químico que se en...,sherlock_holmes
267,No existe una combinación de sucesos que la in...,sherlock_holmes
268,"La soledad es lo que tengo, la soledad es lo q...",sherlock_holmes


In [102]:
df_copy = df.copy()


nlp = spacy.load("es_core_news_sm")

# Función para extraer N-gramas sintácticos
def extract_syntactic_ngrams(text, n=3):
    doc = nlp(text)
    pos_tags = [token.pos_ for token in doc]
    n_grams = zip(*[pos_tags[i:] for i in range(n)])
    return [' '.join(ngram) for ngram in n_grams]

# Función para extraer N-gramas tradicionales
def extract_ngrams(text, n=3):
    tokens = [token.text for token in nlp(text) if not token.is_stop]
    n_grams = zip(*[tokens[i:] for i in range(n)])
    return [' '.join(ngram) for ngram in n_grams]

# Aplicar las funciones anteriores
# df['features'] = df['frases'].apply(extract_syntactic_ngrams)
df['features'] = df['frases'].apply(extract_ngrams)

# Vectorización
vectorizer = CountVectorizer(analyzer=lambda x: x)
X = vectorizer.fit_transform(df['features']).toarray()
y = df['autor']

# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Entrenar el clasificador
# clf = MultinomialNB()
# clf.fit(X_train, y_train)

# entrenar con knn
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X_train, y_train)

# Predecir y evaluar
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.37037037037037035


In [106]:
# cuantos hay de cada autor en el train
print(y_train.value_counts())



jk_rowling         81
sherlock_holmes    70
allan_poe          65
Name: autor, dtype: int64
